In [1]:
# imports
import numpy as np
import pandas as pd

In [45]:
# import and view data
drivers = pd.read_excel('data/drivers.xlsx').set_index('driverId')
lapTimes = pd.read_csv('data/lapTimes.csv')
pitStops = pd.read_csv('data/pitStops.csv')
results = pd.read_csv('data/results.csv')
races = pd.read_csv('data/races.csv')

In [138]:
# get data in form (lap index, driver name/flag, lap time, total time for last n-1 laps, pit stop time) for a single race (969)
# every time a driver finishes a lap, send this data
out = []
RACE_ID = 969

# create elapsed column
lapTimes = lapTimes[lapTimes['raceId'] == RACE_ID].sort_values(['driverId', 'lap'])
lapTimes['elapsed'] = np.zeros(len(lapTimes))
for i in range(len(lapTimes) - 1):
    if lapTimes.iloc[i]['lap'] == 1:
        lapTimes.iloc[i, lapTimes.columns.get_loc('elapsed')] = 0
    lapTimes.iloc[i+1, lapTimes.columns.get_loc('elapsed')] = \
    lapTimes.iloc[i, lapTimes.columns.get_loc('elapsed')] + lapTimes.iloc[i, lapTimes.columns.get_loc('milliseconds')] / 1000

# extract data of interest
lap_data = lapTimes[lapTimes['raceId'] == RACE_ID].sort_values(['lap', 'time'])
pit_data = pitStops[pitStops['raceId'] == RACE_ID]

for (index, row) in lap_data.iterrows():
    name = drivers.loc[row['driverId']]['forename'] + ' ' + drivers.loc[row['driverId']]['surname'] + ' ' + drivers.loc[row['driverId']]['flag']
    if len(pit_stop := pit_data[(pit_data['driverId'] == row['driverId']) & (pit_data['lap'] == row['lap'])]) == 0:
        pit_stop_time = 0
    else:
        pit_stop_time = pit_stop['milliseconds'] / 1000
    
    out.append(
        {
            'lap_index': row['lap'],
            'driver_name': name,
            'lap_time': row['milliseconds'] / 1000,
            'pit_stop_time': pit_stop_time,
            'prev_elapsed': row['elapsed'],
            'position': row['position']
        }
    )
out = sorted(out, key=lambda d: d['prev_elapsed'])

In [137]:
# to excel
pd.DataFrame(out).to_excel('data/out.xlsx')